In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.utils.data import DataLoader, Dataset

import pandas as pd
from tqdm.notebook import tqdm

batch_size = 32
learning_rate = 0.0002
num_epoch = 100

In [2]:
titanic_train = pd.read_csv('./datasets/train.csv')
titanic_test = pd.read_csv('./datasets/test.csv')

titanic_train = titanic_train.loc[:, ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']]

In [3]:
from sklearn import preprocessing
import numpy as np

titanic_train['Sex'] = np.where(titanic_train['Sex'] == 'male', 1, 0)
titanic_train['Age'] = titanic_train['Age'].fillna(titanic_train['Age'].mean())

x = titanic_train.values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
titanic_train = pd.DataFrame(x_scaled, columns=titanic_train.columns)
titanic_train

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,0.000000,1.0,1.0,0.271174,0.125,0.000000,0.014151,0.0
1,0.001124,0.0,0.0,0.472229,0.125,0.000000,0.139136,1.0
2,0.002247,1.0,0.0,0.321438,0.000,0.000000,0.015469,1.0
3,0.003371,0.0,0.0,0.434531,0.125,0.000000,0.103644,1.0
4,0.004494,1.0,1.0,0.434531,0.000,0.000000,0.015713,0.0
...,...,...,...,...,...,...,...,...
886,0.995506,0.5,1.0,0.334004,0.000,0.000000,0.025374,0.0
887,0.996629,0.0,0.0,0.233476,0.000,0.000000,0.058556,1.0
888,0.997753,1.0,0.0,0.367921,0.125,0.333333,0.045771,0.0
889,0.998876,0.0,1.0,0.321438,0.000,0.000000,0.058556,1.0


In [4]:
titanic_train

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,0.000000,1.0,1.0,0.271174,0.125,0.000000,0.014151,0.0
1,0.001124,0.0,0.0,0.472229,0.125,0.000000,0.139136,1.0
2,0.002247,1.0,0.0,0.321438,0.000,0.000000,0.015469,1.0
3,0.003371,0.0,0.0,0.434531,0.125,0.000000,0.103644,1.0
4,0.004494,1.0,1.0,0.434531,0.000,0.000000,0.015713,0.0
...,...,...,...,...,...,...,...,...
886,0.995506,0.5,1.0,0.334004,0.000,0.000000,0.025374,0.0
887,0.996629,0.0,0.0,0.233476,0.000,0.000000,0.058556,1.0
888,0.997753,1.0,0.0,0.367921,0.125,0.333333,0.045771,0.0
889,0.998876,0.0,1.0,0.321438,0.000,0.000000,0.058556,1.0


In [5]:
train_X = titanic_train.loc[:, ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]
train_Y = titanic_train.loc[:, ['Survived']]

In [6]:
train_X

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,0.000000,1.0,1.0,0.271174,0.125,0.000000,0.014151
1,0.001124,0.0,0.0,0.472229,0.125,0.000000,0.139136
2,0.002247,1.0,0.0,0.321438,0.000,0.000000,0.015469
3,0.003371,0.0,0.0,0.434531,0.125,0.000000,0.103644
4,0.004494,1.0,1.0,0.434531,0.000,0.000000,0.015713
...,...,...,...,...,...,...,...
886,0.995506,0.5,1.0,0.334004,0.000,0.000000,0.025374
887,0.996629,0.0,0.0,0.233476,0.000,0.000000,0.058556
888,0.997753,1.0,0.0,0.367921,0.125,0.333333,0.045771
889,0.998876,0.0,1.0,0.321438,0.000,0.000000,0.058556


In [7]:
class TitanicDataset(Dataset):
    def __init__(self, data, labels):
        super().__init__()
        self.data = data.values
        self.labels = labels.values
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.data[idx])
        y = torch.FloatTensor(self.labels[idx])
        return x, y

In [8]:
test_X = titanic_test.loc[:, ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]

train_loader = DataLoader(dataset=TitanicDataset(train_X, train_Y), batch_size=batch_size,
                          shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(test_X, batch_size=batch_size,
                         shuffle=False, num_workers=2, drop_last=True)

In [30]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc_layer = nn.Sequential(
            nn.Linear(7, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        out = self.fc_layer(x)
        return out

In [31]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Network().to(device)
loss_func = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [32]:
loss_arr = []
for i in tqdm(range(num_epoch)):
    loss = 0
    for [data, label] in train_loader:
        x = data.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
    if i % 10 == 0:
        print(loss)

tensor(0.5625, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.5938, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.5312, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.6562, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.6250, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.5312, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.6250, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.5625, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.4688, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.4375, device='cuda:0', grad_fn=<L1LossBackward>)



In [33]:
correct = 0
total = 0

with torch.no_grad():
    for data, label in tqdm(train_loader):
        x = data.to(device)
        y_ = label.to(device)
        
        output = model.forward(x)
        
        total += label.size(0)
        correct += (output == y_).sum().float()
    print("Accuracy of Test Data: {:.4f}%".format(100*correct/total))


Accuracy of Test Data: 38.1944%
